# AGIT Conference

# First Steps in accessing Satellite Imagery on Copernicus Data Space Ecosystem with Sentinel Hub APIs

The Sentinel Hub API is a RESTful API interface that provides access to various satellite imagery archives. It allows you to access raw satellite data, rendered images, statistical analysis, and other features.

In [ ]:
# Utilities
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import geopandas as gpd
import contextily as cx
import getpass

from sentinelhub import (
    SHConfig,
    DataCollection,
    SentinelHubRequest,
    SentinelHubStatistical,
    SentinelHubStatisticalDownloadClient,
    BBox,
    bbox_to_dimensions,
    CRS,
    MimeType,
    Geometry,
    parse_time,
)

from utils import plot_image

## Authentication

You can obtain the credentials for the Sentinel Hub services (`client_id` & `client_secret`) in your [Dashboard](https://shapps.dataspace.copernicus.eu/dashboard/#/). In the user settings, you can create a new OAuth client to generate these credentials. You can find more detailed instructions on the corresponding [documentation page](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Overview/Authentication.html).

Now that you have your `client_id` & `client_secret`, it is recommended to configure a new profile in your Sentinel Hub Python package. Instructions for configuring your Sentinel Hub Python package can be found [here](https://sentinelhub-py.readthedocs.io/en/latest/configure.html). Using these instructions, you can create a profile specifically tailored to use the package to access the Copernicus Data Space Ecosystem data collections. This is useful because changes to the Config class in your notebook are usually only temporary. If you save the configuration in your profile, you do not have to generate new credentials or overwrite/change the default profile every time you start or write a new Jupyter notebook.

If you are using the Sentinel Hub Python package for the Copernicus Data Space Ecosystem for the first time, you should create a profile specifically for the Copernicus Data Space Ecosystem. You can do this in the following cell:

In [ ]:
# Only run this cell if you have not created a configuration.

# config = SHConfig()
# config.sh_client_id = getpass.getpass("Enter your SentinelHub client id")
# config.sh_client_secret = getpass.getpass("Enter your SentinelHub client secret")
config.sh_token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
config.sh_base_url = "https://sh.dataspace.copernicus.eu"
# config.save("cdse")

However, if you have already configured a profile in Sentinel Hub Python for the Copernicus Data Space Ecosystem, then you can run the below cell entering the profile name as a string replacing `<profile_name>`.

In [ ]:
config = SHConfig("cdse")

## Setting an area of interest

The bounding box in `WGS84` coordinate system is `[(longitude and latitude coordinates of lower left and upper right corners)]`. You can get the bbox for a different area at the [bboxfinder](http://bboxfinder.com/) website.

All requests require a bounding box to be given as an instance of `sentinelhub.geometry.BBox` with corresponding Coordinate Reference System (`sentinelhub.constants.CRS`). In our case it is in WGS84 and we can use the predefined WGS84 coordinate reference system from `sentinelhub.constants.CRS`.

In [ ]:
aoi_coords_wgs84 = [12.901121, 47.710216, 13.181293, 47.875821]

When the bounding box bounds have been defined, you can initialize the `BBox` of the area of interest. Using the `bbox_to_dimensions` utility function, you can provide the desired resolution parameter of the image in meters and obtain the output image shape.

In [ ]:
resolution = 10
aoi_bbox = BBox(bbox=aoi_coords_wgs84, crs=CRS.WGS84)
aoi_size = bbox_to_dimensions(aoi_bbox, resolution=resolution)

print(f"Image shape at {resolution} m resolution: {aoi_size} pixels")

## Process API

### Example 1: True Color Image

We build the request according to the [API Reference](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/ApiReference.html), using the `SentinelHubRequest` class. Each Process API request also needs an [evalscript](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Evalscript.html). An evalscript (or "custom script") is a piece of Javascript code which defines how the satellite data shall be processed by Sentinel Hub and what values the service shall return. It is a required part of any [process](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Process.html), [batch processing](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Batch.html) or [OGC request](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/OGC.html).

The information that we specify in the `SentinelHubRequest` object is:
- an evalscript,
- a list of input data collections with time interval,
- a format of the response,
- a bounding box and its size (size or resolution).
- `mosaickingOrder` (optional): in this example we have used `leastCC` which will return pixels from the least cloudy acquisition in the specified time period.

The evalscript in the example is used to select the appropriate bands. We return the RGB (B04, B03, B02) Sentinel-2 L2A bands.

The least cloudy image from the time period is downloaded. Without any additional parameters in the evalscript, the downloaded data will correspond to reflectance values in `UINT8` format (values in 0-255 range).

In [ ]:
evalscript_true_color = """
    //VERSION=3

    function setup() {
        return {
            input: [{
                bands: ["B02", "B03", "B04"]
            }],
            output: {
                bands: 3
            }
        };
    }

    function evaluatePixel(sample) {
        return [sample.B04, sample.B03, sample.B02];
    }
"""

request_true_color = SentinelHubRequest(
    evalscript=evalscript_true_color,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L2A.define_from(
                name="s2l2a", service_url="https://sh.dataspace.copernicus.eu"
            ),
            time_interval=("2025-05-01", "2025-05-20"),
            # other_args={"dataFilter": {"mosaickingOrder": "leastCC"}},
        )
    ],
    responses=[SentinelHubRequest.output_response("default", MimeType.PNG)],
    bbox=aoi_bbox,
    size=aoi_size,
    config=config,
)

The method `get_data()` will always return a list of length 1 with the available image from the requested time interval in the form of numpy arrays.

In [ ]:
true_color_imgs = request_true_color.get_data()

In [ ]:
print(
    f"Returned data is of type = {type(true_color_imgs)} and length {len(true_color_imgs)}."
)
print(
    f"Single element in the list is of type {type(true_color_imgs[-1])} and has shape {true_color_imgs[-1].shape}"
)

In [ ]:
image = true_color_imgs[0]
print(f"Image type: {image.dtype}")

# plot function
# factor 1/255 to scale between 0-1
# factor 3.5 to increase brightness
plot_image(image, factor=3.5 / 255, clip_range=(0, 1))

### Example 2: NDVI Image

Secondly, we will also show you an example of how to calculate and visualise NDVI using the same API. NDVI is a very commonly used spectral vegetation index for vegetation monitoring, for example, monitoring crop growth and yields. As you will notice in the codeblock below, the evalscript has changed substantially:
- we are only using Band 4 and Band 8 as an input into our script.
- In the `evaluatePixel()` function, we calculate NDVI and visualise this using the `imgVals` array. 

In [ ]:
evalscript_ndvi = """
//VERSION=3
function setup() {
  return {
    input: [{
      bands: [
        "B04",
        "B08",
        "dataMask"
      ]
    }],
    output: {
      bands: 4
    }
  }
}
  

function evaluatePixel(sample) {
    let val = (sample.B08 - sample.B04) / (sample.B08 + sample.B04);
    let imgVals = null;
    
    if (val<-1.1) imgVals = [0,0,0];
    else if (val<-0.2) imgVals = [0.75,0.75,0.75];
    else if (val<-0.1) imgVals = [0.86,0.86,0.86];
    else if (val<0) imgVals = [1,1,0.88];
    else if (val<0.025) imgVals = [1,0.98,0.8];
    else if (val<0.05) imgVals = [0.93,0.91,0.71];
    else if (val<0.075) imgVals = [0.87,0.85,0.61];
    else if (val<0.1) imgVals = [0.8,0.78,0.51];
    else if (val<0.125) imgVals = [0.74,0.72,0.42];
    else if (val<0.15) imgVals = [0.69,0.76,0.38];
    else if (val<0.175) imgVals = [0.64,0.8,0.35];
    else if (val<0.2) imgVals = [0.57,0.75,0.32];
    else if (val<0.25) imgVals = [0.5,0.7,0.28];
    else if (val<0.3) imgVals = [0.44,0.64,0.25];
    else if (val<0.35) imgVals = [0.38,0.59,0.21];
    else if (val<0.4) imgVals = [0.31,0.54,0.18];
    else if (val<0.45) imgVals = [0.25,0.49,0.14];
    else if (val<0.5) imgVals = [0.19,0.43,0.11];
    else if (val<0.55) imgVals = [0.13,0.38,0.07];
    else if (val<0.6) imgVals = [0.06,0.33,0.04];
    else imgVals = [0,0.27,0];
    
    
    imgVals.push(sample.dataMask)
    
    return imgVals
}
"""

request_ndvi_img = SentinelHubRequest(
    evalscript=evalscript_ndvi,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L2A.define_from(
                name="s2l2a", service_url="https://sh.dataspace.copernicus.eu"
            ),
            time_interval=("2025-05-01", "2025-05-20"),
            other_args={"dataFilter": {"mosaickingOrder": "leastCC"}},
        )
    ],
    responses=[SentinelHubRequest.output_response("default", MimeType.PNG)],
    bbox=aoi_bbox,
    size=aoi_size,
    config=config,
)

The same method as before is used to request and then visualise the data. In the visualisation, the lighter greens indicate a higher NDVI value (vegetation, forest) and the darker greens (urban areas and water bodies) represent areas with lower NDVI values.

In [ ]:
ndvi_img = request_ndvi_img.get_data()

In [ ]:
print(
    f"Returned data is of type = {type(true_color_imgs)} and length {len(true_color_imgs)}."
)
print(
    f"Single element in the list is of type {type(true_color_imgs[-1])} and has shape {true_color_imgs[-1].shape}"
)

In [ ]:
image = ndvi_img[0]
print(f"Image type: {image.dtype}")

# plot function
plot_image(image, factor=1 / 255)

## Visualising CLMS datasets

In [ ]:
lai_evalscript = """
//VERSION=3
const factor = 1 / 30; // EDIT FACTOR
const offset = 0; // EDIT OFFSET

function setup() {
  return {
    // EDIT VARIABLE NAME
    input: ["LAI", "dataMask"],
    output: [
      { id: "default", bands: 4, sampleType: "UINT8" },
      { id: "index", bands: 1, sampleType: "FLOAT32" },
      { id: "eobrowserStats", bands: 2, sampleType: "FLOAT32" },
      { id: "dataMask", bands: 1 },
    ],
  };
}

function evaluatePixel(samples) {
  // EDIT VARIABLE NAME
  var originalValue = samples.LAI;

  let val = originalValue * factor + offset;

  let dataMask = samples.dataMask;

  const indexVal = dataMask === 1 ? val : NaN;
  const imgVals = visualizer.process(val);

  return {
    default: imgVals.concat(dataMask * 255),
    index: [indexVal],
    eobrowserStats: [val, dataMask],
    dataMask: [dataMask],
  };
}

// EDIT COLOR BAR
const ColorBar = [
  [0.0, [140, 92, 8]],
  [0.03333333333333333, [141, 94, 7]],
  [0.06666666666666667, [143, 97, 7]],
  [0.1, [145, 100, 7]],
  [0.13333333333333333, [147, 102, 7]],
  [0.16666666666666666, [149, 105, 7]],
  [0.2, [151, 108, 7]],
  [0.23333333333333334, [153, 111, 7]],
  [0.26666666666666666, [155, 113, 6]],
  [0.3, [157, 116, 6]],
  [0.3333333333333333, [159, 119, 6]],
  [0.36666666666666664, [161, 121, 6]],
  [0.4, [163, 124, 6]],
  [0.43333333333333335, [164, 127, 6]],
  [0.4666666666666667, [166, 130, 6]],
  [0.5, [168, 132, 6]],
  [0.5333333333333333, [170, 135, 5]],
  [0.5666666666666667, [172, 138, 5]],
  [0.6, [174, 140, 5]],
  [0.6333333333333333, [176, 143, 5]],
  [0.6666666666666666, [178, 146, 5]],
  [0.7, [180, 149, 5]],
  [0.7333333333333333, [182, 151, 5]],
  [0.7666666666666666, [184, 154, 4]],
  [0.8, [186, 157, 4]],
  [0.8333333333333334, [187, 159, 4]],
  [0.8666666666666667, [189, 162, 4]],
  [0.9, [191, 165, 4]],
  [0.9333333333333333, [193, 168, 4]],
  [0.9666666666666667, [195, 170, 4]],
  [1.0, [197, 173, 4]],
  [1.0333333333333332, [199, 176, 3]],
  [1.0666666666666667, [201, 178, 3]],
  [1.1, [203, 181, 3]],
  [1.1333333333333333, [205, 184, 3]],
  [1.1666666666666667, [207, 187, 3]],
  [1.2, [209, 189, 3]],
  [1.2333333333333334, [210, 192, 3]],
  [1.2666666666666666, [212, 195, 2]],
  [1.3, [214, 197, 2]],
  [1.3333333333333333, [216, 200, 2]],
  [1.3666666666666667, [218, 203, 2]],
  [1.4, [220, 206, 2]],
  [1.4333333333333333, [222, 208, 2]],
  [1.4666666666666666, [224, 211, 2]],
  [1.5, [226, 214, 2]],
  [1.5333333333333332, [228, 216, 1]],
  [1.5666666666666667, [230, 219, 1]],
  [1.6, [232, 222, 1]],
  [1.6333333333333333, [233, 225, 1]],
  [1.6666666666666667, [235, 227, 1]],
  [1.7, [237, 230, 1]],
  [1.7333333333333334, [239, 233, 1]],
  [1.7666666666666666, [241, 235, 0]],
  [1.8, [243, 238, 0]],
  [1.8333333333333333, [245, 241, 0]],
  [1.8666666666666667, [247, 244, 0]],
  [1.9, [249, 246, 0]],
  [1.9333333333333333, [251, 249, 0]],
  [1.9666666666666666, [253, 252, 0]],
  [2.0, [255, 255, 0]],
  [2.033333333333333, [250, 254, 0]],
  [2.0666666666666664, [246, 253, 0]],
  [2.1, [242, 252, 0]],
  [2.1333333333333333, [238, 251, 0]],
  [2.1666666666666665, [233, 250, 0]],
  [2.2, [229, 249, 0]],
  [2.2333333333333334, [225, 248, 0]],
  [2.2666666666666666, [221, 247, 0]],
  [2.3, [216, 246, 0]],
  [2.3333333333333335, [212, 245, 0]],
  [2.3666666666666667, [208, 244, 0]],
  [2.4, [204, 244, 0]],
  [2.433333333333333, [199, 243, 0]],
  [2.466666666666667, [195, 242, 0]],
  [2.5, [191, 241, 0]],
  [2.533333333333333, [187, 240, 0]],
  [2.5666666666666664, [182, 239, 0]],
  [2.6, [178, 238, 0]],
  [2.6333333333333333, [174, 237, 0]],
  [2.6666666666666665, [170, 236, 0]],
  [2.7, [165, 235, 0]],
  [2.7333333333333334, [161, 234, 0]],
  [2.7666666666666666, [157, 233, 0]],
  [2.8, [153, 233, 0]],
  [2.8333333333333335, [148, 232, 0]],
  [2.8666666666666667, [144, 231, 0]],
  [2.9, [140, 230, 0]],
  [2.933333333333333, [136, 229, 0]],
  [2.966666666666667, [131, 228, 0]],
  [3.0, [127, 227, 0]],
  [3.033333333333333, [123, 226, 0]],
  [3.0666666666666664, [119, 225, 0]],
  [3.1, [114, 224, 0]],
  [3.1333333333333333, [110, 223, 0]],
  [3.1666666666666665, [106, 222, 0]],
  [3.2, [102, 222, 0]],
  [3.2333333333333334, [97, 221, 0]],
  [3.2666666666666666, [93, 220, 0]],
  [3.3, [89, 219, 0]],
  [3.3333333333333335, [85, 218, 0]],
  [3.3666666666666667, [80, 217, 0]],
  [3.4, [76, 216, 0]],
  [3.433333333333333, [72, 215, 0]],
  [3.466666666666667, [68, 214, 0]],
  [3.5, [63, 213, 0]],
  [3.533333333333333, [59, 212, 0]],
  [3.5666666666666664, [55, 211, 0]],
  [3.6, [51, 211, 0]],
  [3.6333333333333333, [46, 210, 0]],
  [3.6666666666666665, [42, 209, 0]],
  [3.6999999999999997, [38, 208, 0]],
  [3.7333333333333334, [34, 207, 0]],
  [3.7666666666666666, [29, 206, 0]],
  [3.8, [25, 205, 0]],
  [3.8333333333333335, [21, 204, 0]],
  [3.8666666666666667, [17, 203, 0]],
  [3.9, [12, 202, 0]],
  [3.933333333333333, [8, 201, 0]],
  [3.966666666666667, [4, 200, 0]],
  [4.0, [0, 200, 0]],
  [4.033333333333333, [0, 198, 0]],
  [4.066666666666666, [0, 197, 0]],
  [4.1, [0, 196, 0]],
  [4.133333333333333, [0, 195, 0]],
  [4.166666666666667, [0, 194, 0]],
  [4.2, [0, 193, 0]],
  [4.233333333333333, [0, 192, 0]],
  [4.266666666666667, [0, 191, 0]],
  [4.3, [0, 190, 0]],
  [4.333333333333333, [0, 188, 0]],
  [4.366666666666666, [0, 187, 0]],
  [4.4, [0, 186, 0]],
  [4.433333333333334, [0, 185, 0]],
  [4.466666666666667, [0, 184, 0]],
  [4.5, [0, 183, 0]],
  [4.533333333333333, [0, 182, 0]],
  [4.566666666666666, [0, 181, 0]],
  [4.6, [0, 180, 0]],
  [4.633333333333333, [0, 178, 0]],
  [4.666666666666667, [0, 177, 0]],
  [4.7, [0, 176, 0]],
  [4.733333333333333, [0, 175, 0]],
  [4.766666666666667, [0, 174, 0]],
  [4.8, [0, 173, 0]],
  [4.833333333333333, [0, 172, 0]],
  [4.866666666666666, [0, 171, 0]],
  [4.9, [0, 170, 0]],
  [4.933333333333334, [0, 168, 0]],
  [4.966666666666667, [0, 167, 0]],
  [5.0, [0, 166, 0]],
  [5.033333333333333, [0, 165, 0]],
  [5.066666666666666, [0, 164, 0]],
  [5.1, [0, 163, 0]],
  [5.133333333333333, [0, 162, 0]],
  [5.166666666666667, [0, 161, 0]],
  [5.2, [0, 160, 0]],
  [5.233333333333333, [0, 158, 0]],
  [5.266666666666667, [0, 157, 0]],
  [5.3, [0, 156, 0]],
  [5.333333333333333, [0, 155, 0]],
  [5.366666666666666, [0, 154, 0]],
  [5.4, [0, 153, 0]],
  [5.433333333333334, [0, 152, 0]],
  [5.466666666666667, [0, 151, 0]],
  [5.5, [0, 150, 0]],
  [5.533333333333333, [0, 148, 0]],
  [5.566666666666666, [0, 147, 0]],
  [5.6, [0, 146, 0]],
  [5.633333333333333, [0, 145, 0]],
  [5.666666666666667, [0, 144, 0]],
  [5.7, [0, 143, 0]],
  [5.733333333333333, [0, 142, 0]],
  [5.766666666666667, [0, 141, 0]],
  [5.8, [0, 140, 0]],
  [5.833333333333333, [0, 138, 0]],
  [5.866666666666666, [0, 137, 0]],
  [5.9, [0, 136, 0]],
  [5.933333333333334, [0, 135, 0]],
  [5.966666666666667, [0, 134, 0]],
  [6.0, [0, 133, 0]],
  [6.033333333333333, [0, 132, 0]],
  [6.066666666666666, [0, 131, 0]],
  [6.1, [0, 130, 0]],
  [6.133333333333333, [0, 128, 0]],
  [6.166666666666667, [0, 127, 0]],
  [6.2, [0, 126, 0]],
  [6.233333333333333, [0, 125, 0]],
  [6.266666666666667, [0, 124, 0]],
  [6.3, [0, 123, 0]],
  [6.333333333333333, [0, 122, 0]],
  [6.366666666666666, [0, 121, 0]],
  [6.4, [0, 120, 0]],
  [6.433333333333334, [0, 118, 0]],
  [6.466666666666667, [0, 117, 0]],
  [6.5, [0, 116, 0]],
  [6.533333333333333, [0, 115, 0]],
  [6.566666666666666, [0, 114, 0]],
  [6.6, [0, 113, 0]],
  [6.633333333333333, [0, 112, 0]],
  [6.666666666666667, [0, 111, 0]],
  [6.7, [0, 110, 0]],
  [6.733333333333333, [0, 108, 0]],
  [6.766666666666667, [0, 107, 0]],
  [6.8, [0, 106, 0]],
  [6.833333333333333, [0, 105, 0]],
  [6.866666666666666, [0, 104, 0]],
  [6.8999999999999995, [0, 103, 0]],
  [6.933333333333334, [0, 102, 0]],
  [6.966666666666667, [0, 101, 0]],
  [7.0, [0, 100, 0]],
  [7.033333333333333, [255, 255, 255]],
  [8.5, [255, 255, 255]],
];
const visualizer = new ColorRampVisualizer(ColorBar);
"""

In [ ]:
fields_gdf = gpd.read_file("data/austria.geojson")
fields_gdf["area"] = fields_gdf.geometry.area

fields_gdf_proj = fields_gdf.to_crs(epsg=3857)

# Explore the GeoDataFrame, visualizing the 'area' column with a specified colormap
ax = fields_gdf_proj.plot(alpha=0.5, cmap="Paired", edgecolor="k", figsize=(15, 15))
cx.add_basemap(ax)

In [ ]:
geometry = Geometry(geometry=fields_gdf.geometry.values[0], crs=CRS.WGS84)

request = SentinelHubRequest(
    evalscript=lai_evalscript,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.define_byoc(
                "559b2871-ddbd-41f9-8ab0-5242a454e411"
            ),
            time_interval=("2024-06-01", "2024-06-10"),
        ),
    ],
    responses=[
        SentinelHubRequest.output_response("default", MimeType.PNG),
    ],
    geometry=geometry,
    resolution=[0.01, 0.01],
    config=config,
)

response = request.get_data()

In [ ]:
lai_img = request.get_data()

In [ ]:
print(
    f"Returned data is of type = {type(true_color_imgs)} and length {len(true_color_imgs)}."
)
print(
    f"Single element in the list is of type {type(true_color_imgs[-1])} and has shape {true_color_imgs[-1].shape}"
)

In [ ]:
image = lai_img[0]
print(f"Image type: {image.dtype}")

# plot function
plot_image(image, factor=1 / 255)

## Now time to create your own visualisation!

- You can either adapt the evalscript in the examples you have just seen or even create your own request!
- Try changing the time range, the sensor type, different band combinations. Can you work out how to make a false color image?
- You can find evalscripts in the [Custom Scripts Repository](https://custom-scripts.sentinel-hub.com/)